**Instruction**:    
1. Download the ipynb, which you want to convert, on your local computer.    
2. Run the code below to upload the ipynb.  
3. The html version will be downloaded automatically on your local machine.

Enjoy it!

In [ ]:
#@title Convert ipynb to HTML in Colab
# Upload ipynb
from google.colab import files
f = files.upload()

# Convert ipynb to html
import subprocess
file0 = list(f.keys())[0]
_ = subprocess.run(["pip", "install", "nbconvert"])
_ = subprocess.run(["jupyter", "nbconvert", file0, "--to", "html"])

# download the html
files.download(file0[:-5]+"html")


In [ ]:
import pandas as pd

# pd.read_stata('convstate.dta').to_csv("csv_files/convstate.csv", index = False)

# pd.read_stata('countytodma.dta').to_csv("csv_files/countytodma.csv", index = False)

#pd.read_stata('ctyunemrate.dta').to_csv("csv_files/ctyunemrate.csv", index = False
# print("ctyunemrate.dta:")
# print(pd.read_stata('ctyunemrate.dta'))

#pd.read_stata('dmacode.dta').to_csv("csv_files/dmacode.csv", index = False)

# print("dma_characteristics.dta:")
# print(pd.read_stata('dma_characteristics.dta'))

# print("dmacode.dta:")
# print(pd.read_stata('dmacode.dta'))
#pd.read_stata('googletrends-state-raw.dta').to_csv("csv_files/googletrends-state-raw.csv", index = False)

# print("googletrends-state-raw.dta:")
# print(pd.read_stata('googletrends-state-raw.dta'))
#pd.read_stata('googletrends-state.dta').to_csv("csv_files/googletrends-state.csv", index = False)

# print("googletrends-state.dta:")
# print(pd.read_stata('googletrends-state.dta'))
#pd.read_stata('googletrends-state2.dta').to_csv("csv_files/googletrends-state2.csv", index = False)

# print("googletrends-state2.dta:")
# print(pd.read_stata('googletrends-state2.dta'))
pd.read_stata('googletrends-thehills.dta').to_csv("csv_files/googletrends-thehills.csv", index = False)

# print("googletrends-thehills.dta:")
# print(pd.read_stata('googletrends-thehills.dta'))

# print("graph-treatment.do:")
# print(pd.read_stata('graph-treatment.do'))

In [ ]:
import pandas as pd

# Load your main dataset
dataset = pd.read_csv('csv_files/merged_dropped_df2_df3.csv')

# Load the Twitter data
twitter_data = pd.read_csv('csv_files/twitter.csv')


print(dataset.columns)
print(twitter_data.columns)


# Example of renaming columns in the twitter_data DataFrame
twitter_data.rename(columns={
    'Log Tweet Rate: Birth Control': 'birth_control_tweet_rate',
    'Log Tweet Rate: Abortion': 'abortion_tweet_rate'
    # Add more mappings here if you have more columns
}, inplace=True)

# Rename the 'abort' column in your main dataset to match the naming convention
dataset.rename(columns={
    'abort': 'abortion_rate'
    # Add any other column renames here
}, inplace=True)
# Convert to string if these are categorical or identifier variables
dataset['abortion_rate'] = dataset['abortion_rate'].astype(str)
twitter_data['abortion_tweet_rate'] = twitter_data['abortion_tweet_rate'].astype(str)
# Merging datasets on a common column, e.g., 'date'
merged_dataset = pd.merge(dataset, twitter_data, left_on='abortion_rate', right_on='abortion_tweet_rate', how='left')

print(merged_dataset.head())

In [ ]:
import pandas as pd

# Load the datasets
df1 = pd.read_csv('csv_files/googletrends-state-raw.csv')
df2 = pd.read_csv('csv_files/googletrends-state.csv')
df3 = pd.read_csv('csv_files/googletrends-state2.csv')

# Since df2 and df3 share year column
merged_df2_df3 = pd.merge(df2, df3, on= ["stname", "year", "fempop1519", "hgabort", "hgpill", "sexft", "virgin", "sexhurt", "hgbc", "index16p", "abort", "bc"], how="inner")
merged_df2_df3.to_csv("csv_files/merged_df2_df3.csv", index="False")


In [ ]:
import pandas as pd

# Load the datasets
df1 = pd.read_csv('csv_files/googletrends-state-raw.csv')
df2 = pd.read_csv('csv_files/googletrends-state.csv')
df3 = pd.read_csv('csv_files/googletrends-state2.csv')

# Since df2 and df3 share year column
merged_df2_df3 = pd.merge(df2, df3, on= ["stname", "year", "fempop1519", "hgabort", "hgpill", "sexft", "virgin", "sexhurt", "hgbc", "index16p", "abort", "bc"], how="inner")
merged_df2_df3.to_csv("csv_files/merged_df2_df3.csv", index="False")


In [ ]:
import pandas as pd

df = pd.read_csv('csv_files/merged_df2_df3.csv')

merged_dropped_df2_df3 = df.drop(['index16p', 'stview16pTM', 'stview0809', 'strate16pTM', 'strate0809'], axis=1)
merged_dropped_df2_df3.to_csv('csv_files/merged_dropped_df2_df3.csv', index="False")

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS

# Load the dataset
df = pd.read_csv('csv_files/merge_datasets.csv')  # Adjust with your actual dataset path

# Data Cleaning Steps
df.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace infinite values with NaN

# Example: Fill NaN values for 'sexft' and 'birth control: (United States)' with the mean or median
df['sexft'].fillna(df['sexft'].mean(), inplace=True)
df['birth control: (United States)'].fillna(df['birth control: (United States)'].median(), inplace=True)
df['16 and pregnant: (United States)'].fillna(df['16 and pregnant: (United States)'].mean(), inplace=True)

# Continue with your analysis only if df is not empty
if not df.empty:
    # Define variables
    Y = df['abort']
    X = df[['sexft', 'birth control: (United States)']]
    X = sm.add_constant(X)
    Z = df['16 and pregnant: (United States)']

    # Perform IV Estimation
    iv_model = IV2SLS(Y, X, Z).fit()

    # Print the summary of the IV estimation
    print(iv_model.summary())
else:
    print("DataFrame is empty after handling missing values.")

# Assuming 'iv_model' is your IV2SLS model result
summary = iv_model.summary().as_text()

# Specify the path and filename for your results
results_path = 'iv_results.txt'

# Save the summary to a file
with open(results_path, 'w') as f:
    f.write(summary)